In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=72b31fdb333f659491c54dd7efec41affe9693ea1a0bbf70a080e87d0ab87bd4
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import preprocessing
import pandas as pd
import csv
import numpy as np
import emoji


In [ ]:

def read_csv(filename = '/content/drive/My Drive/Colab Notebooks/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
    
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

                                  

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [ ]:
X_train, Y_train= read_csv('/content/drive/My Drive/Colab Notebooks/train_emoji.csv')
X_test, Y_test = read_csv('/content/drive/My Drive/Colab Notebooks/tesss.csv')

In [ ]:
 maxLen = len(max(X_train, key=len).split())
 print(maxLen)

10


In [ ]:
for idx in range(20):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long 😞
I want to go play ⚾
she did not answer my text 😞
Your stupidity has no limit 😞
how many points did he score ⚾
my algorithm performs poorly 😞
I got approved 😄
Stop shouting at me 😞
Sounds like a fun plan ha ha 😄
no one likes him 😞
the game just finished ⚾
I will celebrate soon 😄


In [ ]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [ ]:
idx = 50
print(f"Sentence '{X_train[50]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [1. 0. 0. 0. 0.]


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-06-25 16:30:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-25 16:30:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-25 16:30:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
print('Indexing word vectors.')

embeddings_index = {}
#f = open('glove.6B.50d.txt', encoding='utf-8')
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


Indexing word vectors.


In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [ ]:
word = "potatoes"
idx = 289845
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])
print(word_to_vec_map["cucumber"].shape[0]) 
len(index_to_word)

the index of potatoes in the vocabulary is 289845
the 289845th word in the vocabulary is potatoes
50


400000

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)  

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]
    X_indices=np.zeros((m,max_len))
    for i in range(m):
        sentence=X[i].lower().split()
        j=0
        for w in sentence:
            X_indices[i, j]=word_to_index[w]
            j=+1
    return X_indices        

In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345. 225122.      0.      0.      0.]
 [220930.  69714.      0.      0.      0.]
 [151204. 394475.      0.      0.      0.]]


In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0] 
    emb_matrix=np.zeros((vocab_len,emb_dim))
    for word,idx in word_to_index.items():
      emb_matrix[idx,:]=word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][60000])

weights[0][1][3] = [ 0.52112  -0.69663   0.3253   -1.1044   -1.8012    0.14077   1.6317
 -0.66187   0.86257   0.4609   -1.2081    0.83778   1.5819   -0.64043
 -0.31074  -0.99099   0.74277  -0.14722   0.038262  0.16919   0.23585
 -2.2794    0.62587  -1.1215   -1.6721   -0.22342   0.69452  -0.12559
 -0.69891  -0.12324   0.18451   0.14188  -0.38967   0.14157   1.1081
  1.5815    1.2761    0.23443   0.28287  -1.3671    0.40378   1.3651
 -0.29074  -0.70388  -0.043222 -1.8392    0.55578   0.12606   1.4342
  1.8641  ]


In [ ]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    embedding_layer =pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)  
    X = LSTM(100, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(100)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation(activation='softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model


In [ ]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 100)           60400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
315/315 [==============================] - 1s 2ms/step - loss: 1.5732 - accuracy: 0.2603
Epoch 2/50
315/315 [==============================] - 0s 689us/step - loss: 1.4676 - accuracy: 0.3714
Epoch 3/50
315/315 [==============================] - 0s 669us/step - loss: 1.3085 - accuracy: 0.4476
Epoch 4/50
315/315 [==============================] - 0s 677us/step - loss: 1.0248 - accuracy: 0.6095
Epoch 5/50
315/315 [==============================] - 0s 690us/step - loss: 0.8286 - accuracy: 0.6730
Epoch 6/50
315/315 [==============================] - 0s 677us/step - loss: 0.7284 - accuracy: 0.7270
Epoch 7/50
315/315 [==============================] - 0s 663us/step - loss: 0.6606 - accuracy: 0.7524
Epoch 8/50
315/315 [==============================] - 0s 646us/step - loss: 0.5756 - accuracy: 0.8000
Epoch 9/50
315/315 [==============================] - 0s 701us/step - loss: 0.5460 - accuracy: 0.8000
Epoch 10/50
315/315 [==============================] - 0s 664us/step - loss: 0.4851 

In [ ]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
loss, acc = model.evaluate(X_test_indices,Y_oh_test)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 2ms/step

Test accuracy =  0.8571428656578064


In [ ]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😄 prediction: he is a good friend	❤️
Expected emoji:😞 prediction: work is hard	⚾
Expected emoji:😄 prediction: valentine day is near	❤️
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:❤️ prediction: I like your jacket 	😄
Expected emoji:😞 prediction: I do not want to joke	😄
Expected emoji:🍴 prediction: I did not have breakfast 😞


In [ ]:
x_test = np.array([' fuck you'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

 fuck you 😞


In [ ]:
model.save("/content/drive/My Drive/Colab Notebooks")

OSError: ignored